In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import sys
sys.path.insert(0,'../tools')
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import copy
from multiprocessing import Pool
from sklearn.metrics import roc_curve, auc
import sklearn
import time
import utils
import os
from joblib import dump, load
import imp

In [2]:
imp.reload(utils)
import pickle
text_train,label_train,text_test,label_test = utils.load_raw_data_new()
train_len = len(text_train)
print(train_len)
text_train.extend(text_test)
label_train.extend(label_test)
text_total = text_train
label_total = label_train
new_text_total = utils.back_to_doc(text_total)

2887


In [3]:
src_dir = '../preprocessing/result/ratio2/'
new_text_train = pickle.load(open(src_dir + 'text_train_lemma.pkl','rb'))
new_text_test = pickle.load(open(src_dir + 'text_test_lemma.pkl','rb'))

In [10]:
Tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1),min_df =5,stop_words='english',max_df = 0.8)
X_train = Tfidf_vectorizer.fit_transform(new_text_train)
Y_train = label_total[:train_len]
X_test = Tfidf_vectorizer.transform(new_text_test)
Y_test = label_total[train_len:]
print(np.shape(X_train))

(2887, 11741)


In [11]:
SVC_clf_1gram = svm.SVC(class_weight = 'balanced',kernel = 'linear',probability=1)
SVC_clf_1gram.fit(X_train,Y_train)
y_pred = SVC_clf_1gram.predict(X_test)

y_pred_proba = SVC_clf_1gram.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)

print(metrics.classification_report(Y_test, y_pred, target_names=['No_infection','infection']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

              precision    recall  f1-score   support

No_infection       0.89      0.93      0.91       481
   infection       0.85      0.76      0.80       241

    accuracy                           0.87       722
   macro avg       0.87      0.85      0.85       722
weighted avg       0.87      0.87      0.87       722

[[448  33]
 [ 58 183]]
0.8453645154889968


In [39]:
y_pred_proba = SVC_clf_1gram.predict_proba(X_test)
np.shape(y_pred_proba)
y_pred_proba = y_pred_proba[:,1]
Y_test  = np.array(Y_test)
diff = Y_test - y_pred_proba
obivious_wrong = np.where(diff < -0.5)[0]
print(len(obivious_wrong))

33


In [42]:
obivious_wrong_bert = np.array([ 38,  44,  50,  60,  71, 127, 137, 154, 163, 165, 246, 270, 302,
       311, 329, 393, 395, 449, 461, 489, 513, 522, 523, 552, 561, 592,
       622, 625, 662, 666, 705, 712])

In [47]:
obivious_wrong_bert = set(obivious_wrong_bert)
obivious_wrong = set(obivious_wrong)
both_wrong = obivious_wrong.intersection(obivious_wrong_bert)

In [48]:
len(obivious_wrong)

33

In [49]:
len(both_wrong)

18

In [50]:
both_wrong

{38,
 44,
 60,
 71,
 137,
 270,
 311,
 329,
 393,
 449,
 461,
 513,
 523,
 592,
 625,
 662,
 666,
 705}

In [40]:
log = 'The label accroding to ICD is {}, but model predicted: {}'
for sample_index in obivious_wrong:
    print(log.format(Y_test[sample_index], y_pred_proba[sample_index]))

The label accroding to ICD is 0, but model predicted: 0.7212356043702999
The label accroding to ICD is 0, but model predicted: 0.7036312632139125
The label accroding to ICD is 0, but model predicted: 0.521614449713669
The label accroding to ICD is 0, but model predicted: 0.7127888199000468
The label accroding to ICD is 0, but model predicted: 0.864576265642625
The label accroding to ICD is 0, but model predicted: 0.6552214853689756
The label accroding to ICD is 0, but model predicted: 0.5577633759402486
The label accroding to ICD is 0, but model predicted: 0.6657360482573027
The label accroding to ICD is 0, but model predicted: 0.9555607643441474
The label accroding to ICD is 0, but model predicted: 0.70812010553895
The label accroding to ICD is 0, but model predicted: 0.5750495120824908
The label accroding to ICD is 0, but model predicted: 0.613327488997818
The label accroding to ICD is 0, but model predicted: 0.9627544899924041
The label accroding to ICD is 0, but model predicted: 0.

In [ ]:
doc_to_review = {38, 44, 60, 71, 137, 270, 311, 329, 393, 449, 461, 513, 523, 592, 625, 662, 666, 705}
att_generator = utils.output_att_scores(hpara1,model_word,model_sent,save_dir,data_generator,tokenizer,use_angular=False,narrow=True,epoch=16)